In [1]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Activation
from sklearn import datasets
from sklearn.preprocessing import minmax_scale, LabelBinarizer
sys.path.append('../../generative-adversarial-nets')
from ganetwork import GAN, CGAN

Using TensorFlow backend.


In [2]:
ds = datasets.load_iris()
X, y = minmax_scale(ds.data), LabelBinarizer().fit_transform(ds.target)

In [3]:
gan_generator = Sequential([
    Dense(input_dim=10, output_dim=20),
    Activation('tanh'),
    Dense(output_dim=10),
    Activation('tanh'),
    Dense(output_dim=X.shape[1]),
    Activation('sigmoid')
])

gan_discriminator = Sequential([
    Dense(input_dim=X.shape[1], output_dim=20),
    Activation('tanh'),
    Dense(output_dim=10),
    Activation('tanh'),
    Dense(1),
    Activation('sigmoid')
])

gan = GAN(gan_generator, gan_discriminator)
gan.train(X, nb_epoch=10, batch_size=5, verbose=1)

Epoch 0, d_loss: 0.707718, g_loss: 0.656318
Epoch 1, d_loss: 0.722398, g_loss: 0.674569
Epoch 2, d_loss: 0.702905, g_loss: 0.688403
Epoch 3, d_loss: 0.663163, g_loss: 0.690247
Epoch 4, d_loss: 0.639798, g_loss: 0.716955
Epoch 5, d_loss: 0.695569, g_loss: 0.654570
Epoch 6, d_loss: 0.708293, g_loss: 0.646714
Epoch 7, d_loss: 0.700645, g_loss: 0.696907
Epoch 8, d_loss: 0.684038, g_loss: 0.697009
Epoch 9, d_loss: 0.703230, g_loss: 0.720268


In [ ]:
cgan_generator = Sequential([
    Dense(input_dim=10, output_dim=20),
    Activation('tanh'),
    Dense(output_dim=10),
    Activation('tanh'),
    Dense(output_dim=X.shape[1]),
    Activation('sigmoid')
])

cgan_discriminator = Sequential([
    Dense(input_dim=(X.shape[1] + y.shape[1]), output_dim=20),
    Activation('tanh'),
    Dense(output_dim=10),
    Activation('tanh'),
    Dense(1),
    Activation('sigmoid')
])

cgan = CGAN(cgan_generator, cgan_discriminator)
cgan.train(X, y, nb_epoch=10, batch_size=5, verbose=1)